In [215]:
import pulp

In [227]:

soc = [20,20,20,20]
n = len(soc)
stations = [20,30,50]
speed = 10
charging_rate_station = 10
ex_charging_rate_bess = 5
arrival_time = [1,2,3,4]
T = 1000
bess_capacity = 50

In [228]:
#initiate the model
cs = pulp.LpProblem("station",pulp.LpMinimize)
vehicles=[]
#add variables
soc_vehicle = {}
arrival_time_vehicle = {}
for i in range(n):
    vehicles.append(f"vehicle{i}")
    soc_vehicle[f"vehicle{i}"] = soc[i]
    arrival_time_vehicle[f"vehicle{i}"] = arrival_time[i]
M = 100000
print(vehicles)
#arrival time at stations
a1 = pulp.LpVariable.dict("Arrival Time 1",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
a2 = pulp.LpVariable.dict("Arrival Time 2",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
# servicing time at stations
s1 = pulp.LpVariable.dict("Service Time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
s2 = pulp.LpVariable.dict("Service Time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
#initial charging time at sations
u1 = pulp.LpVariable.dict("initial charge time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
u2 = pulp.LpVariable.dict("initial charge time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
#final charge time at stations
d1 = pulp.LpVariable.dict("final charge time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
d2 = pulp.LpVariable.dict("final charge time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
# sigma- temporal ordering of vehicles
o1 = pulp.LpVariable.dict("sigma 1",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
o2 = pulp.LpVariable.dict("sigma 2",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
# to see if vehicle is gonna use the respective station
sb1 = pulp.LpVariable.dict("Service Time 1 binary",[vehicle for vehicle in vehicles],cat=pulp.LpBinary)
sb2 = pulp.LpVariable.dict("Service Time 2 binary",[vehicle for vehicle in vehicles],cat=pulp.LpBinary)
#bess servicing times at stations
b1 = pulp.LpVariable.dict("BESS service Time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
b2 = pulp.LpVariable.dict("BESS service Time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
# charge in bess available for vehicles
bc1 = pulp.LpVariable.dict("BESS charge 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
bc2 = pulp.LpVariable.dict("BESS charge 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
# if bess reached max charge or not
charge1 = pulp.LpVariable.dict("reached max charge 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpBinary)
charge2 = pulp.LpVariable.dict("reached max charge 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpBinary)
#amount to charge
bac1 = pulp.LpVariable.dict("BESS charge for vehicle 1",[vehicle for vehicle in vehicles],cat=pulp.LpInteger)
bac2 = pulp.LpVariable.dict("BESS charge for vehicle 2",[vehicle for vehicle in vehicles],cat=pulp.LpInteger)





['vehicle0', 'vehicle1', 'vehicle2', 'vehicle3']


In [229]:
#objective function
cs += pulp.lpSum([d1[vehicle]-a1[vehicle] + d2[vehicle]-a2[vehicle]] for vehicle in vehicles)
#blending+= (pulp.lpSum([4.32*ing_weight[(i,"pork")]+2.46*ing_weight[(i,"wheat")]+1.86*ing_weight[(i,"starch")] for i in sausages]))

In [230]:
#constraints
for vehicle in vehicles:
    #departure = service + charge start
    cs += d1[vehicle] == s1[vehicle]+u1[vehicle]
    cs += d2[vehicle] == s2[vehicle]+u2[vehicle]
    
    # charge start > arival time
    cs += u1[vehicle]>=a1[vehicle]
    cs += u2[vehicle]>=a2[vehicle]
    cs += u2[vehicle]<=T-s2[vehicle]
    
    # arrival times of each station
    cs += a1[vehicle] == arrival_time_vehicle[vehicle]+(stations[0]/speed)
    cs += a2[vehicle] == (stations[1]/speed) + d1[vehicle]
    
    # ensuring soc capabilities
    cs += charging_rate_station*s1[vehicle]+ex_charging_rate_bess*b1[vehicle] + soc_vehicle[vehicle]>=stations[0]+stations[1]
    cs += charging_rate_station*s1[vehicle]+ex_charging_rate_bess*b1[vehicle] + soc_vehicle[vehicle]<=100
    cs += charging_rate_station*(s2[vehicle]+s1[vehicle])+ex_charging_rate_bess*(b2[vehicle]+b1[vehicle])+soc_vehicle[vehicle]>=stations[0]+stations[1]+stations[2]
    cs += charging_rate_station*(s2[vehicle]+s1[vehicle])+ex_charging_rate_bess*(b2[vehicle]+b1[vehicle])+soc_vehicle[vehicle]-stations[0]-stations[1]<=100
    
    #to see if a vehicle wants to cross the station or charge there
    cs += M*sb1[vehicle] >= s1[vehicle]
    cs += M*sb1[vehicle] >=s1[vehicle]
    cs += sb1[vehicle] <= s1[vehicle]
    cs += M*sb2[vehicle] >=s2[vehicle]
    cs += sb2[vehicle] <= s2[vehicle]
    # bess time should be less than total charging time at 
    cs += b1[vehicle]<=s1[vehicle]
    cs += b2[vehicle]<=s2[vehicle]
    cs += b1[vehicle]*ex_charging_rate_bess<=bc1[vehicle]
    cs += b2[vehicle]*ex_charging_rate_bess<=bc2[vehicle]
    #checking if the amount we charge is greater than 100 or not
    cs += M*charge1[vehicle]>=bac1[vehicle]
    cs += M*(1-charge1[vehicle]) >= -bac1[vehicle]
    cs += M*charge2[vehicle]>=bac2[vehicle]
    cs += M*(1-charge2[vehicle]) >= -bac2[vehicle]
    # implementing max of bess capaicty
    cs += bc1[vehicle]>= bess_capacity - M*(1-charge1[vehicle])
    cs += bc1[vehicle]>= bess_capacity + bac1[vehicle] - M*(charge1[vehicle])
    cs += bc1[vehicle]<= bess_capacity + M*(1-charge1[vehicle])
    cs += bc1[vehicle]<= bess_capacity+bac1[vehicle] + M*(charge1[vehicle])  
     
    cs += bc2[vehicle]>= bess_capacity - M*(1-charge2[vehicle])
    cs += bc2[vehicle]>= bess_capacity + bac2[vehicle] - M*(charge2[vehicle])
    cs += bc2[vehicle]<= bess_capacity + M*(1-charge2[vehicle])
    cs += bc2[vehicle]<= bess_capacity+bac2[vehicle] + M*(charge2[vehicle])

# for bess implementations
for i in range(len(vehicles)):
    cs += bac1[vehicles[i]] ==  (u1[vehicles[i]]-u1[vehicles[0]] - 2*pulp.lpSum([b1[vehicles[j]]] for j in range(i-1,-1,-1)))*ex_charging_rate_bess
    cs += bac2[vehicles[i]] == (u2[vehicles[i]]-u2[vehicles[0]] - 2*pulp.lpSum([b2[vehicles[j]]] for j in range(i-1,-1,-1)))*ex_charging_rate_bess
# for waiting time implementation 
for vehicle1 in vehicles:
    
    for vehicle2 in vehicles:
        if vehicle1!=vehicle2:
            
            #making sure o1[vehicle1+vehicle2] = 1 when vehicle1 in front of vehicle2
            cs += M*o1[vehicle1+vehicle2] >= a1[vehicle2]-a1[vehicle1]
            cs += M*(1-o1[vehicle1+vehicle2])>= a1[vehicle1]-a1[vehicle2]
            cs += M*o2[vehicle1+vehicle2] >= a2[vehicle2]-a2[vehicle1]
            cs += M*(1-o2[vehicle1+vehicle2]) >= a2[vehicle1]-a2[vehicle2]
            
            #implementing wait time
            cs += u1[vehicle2]-u1[vehicle1]-s1[vehicle1]>=(o1[vehicle1+vehicle2]-1)*T-M*(1-sb1[vehicle2])
            cs += u2[vehicle2]-u2[vehicle1]-s2[vehicle1]>=(o2[vehicle1+vehicle2]-1)*T-M*(1-sb2[vehicle2])
            
            #making sure only one vehicle is front of another
            cs += o1[vehicle1+vehicle2] + o1[vehicle2+vehicle1] >= 1
            cs += o2[vehicle1+vehicle2] + o2[vehicle2+vehicle1] >= 1




In [231]:
cs

station:
MINIMIZE
-1*Arrival_Time_1_vehicle0 + -1*Arrival_Time_1_vehicle1 + -1*Arrival_Time_1_vehicle2 + -1*Arrival_Time_1_vehicle3 + -1*Arrival_Time_2_vehicle0 + -1*Arrival_Time_2_vehicle1 + -1*Arrival_Time_2_vehicle2 + -1*Arrival_Time_2_vehicle3 + 1*final_charge_time_1_vehicle0 + 1*final_charge_time_1_vehicle1 + 1*final_charge_time_1_vehicle2 + 1*final_charge_time_1_vehicle3 + 1*final_charge_time_2_vehicle0 + 1*final_charge_time_2_vehicle1 + 1*final_charge_time_2_vehicle2 + 1*final_charge_time_2_vehicle3 + 0
SUBJECT TO
_C1: - Service_Time_1_vehicle0 + final_charge_time_1_vehicle0
 - initial_charge_time_1_vehicle0 = 0

_C2: - Service_Time_2_vehicle0 + final_charge_time_2_vehicle0
 - initial_charge_time_2_vehicle0 = 0

_C3: - Arrival_Time_1_vehicle0 + initial_charge_time_1_vehicle0 >= 0

_C4: - Arrival_Time_2_vehicle0 + initial_charge_time_2_vehicle0 >= 0

_C5: Service_Time_2_vehicle0 + initial_charge_time_2_vehicle0 <= 1000

_C6: Arrival_Time_1_vehicle0 = 3

_C7: Arrival_Time_2_vehicl

In [232]:
cs.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sushi/Desktop/internship/Routing/.venv/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/05b290a420c44904a9e82f6284e1dc7a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/05b290a420c44904a9e82f6284e1dc7a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 237 COLUMNS
At line 1084 RHS
At line 1317 BOUNDS
At line 1414 ENDATA
Problem MODEL has 232 rows, 96 columns and 636 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 21.3333 - 0.00 seconds
Cgl0003I 4 fixed, 50 tightened bounds, 35 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 15 tightened bounds, 20 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0004I processed model has 127 rows, 62 columns (62 integer (22 of which binary)) and 36

1

In [233]:
pulp.LpStatus[cs.status]

'Optimal'

In [247]:
for vehicle in vehicles:
    print(f"{vehicle} waits at s1 for {u1[vehicle].varValue-a1[vehicle].varValue}, s2: for {u2[vehicle].varValue-a2[vehicle].varValue}")
    print(f"{vehicle} charges at s1 for {b1[vehicle].varValue} using the bess and station, and for {s1[vehicle].varValue-b1[vehicle].varValue} only using station")
    print(f"{vehicle} charges at s2 for {b2[vehicle].varValue} using the bess and station, and for {s2[vehicle].varValue-b2[vehicle].varValue} only using station")
    print(f"{vehicle} reaches s1 , s2 at {a1[vehicle].varValue, a2[vehicle].varValue}")
    print(f"bess energy level for {vehicle} is {bc1[vehicle].varValue},{bc2[vehicle].varValue}")
    print(f"{vehicle} charges s1 , s2 at {u1[vehicle].varValue, u2[vehicle].varValue}")
    print(f"{vehicle} charge at s1 , s2 at {charge1[vehicle].varValue, charge2[vehicle].varValue}")

    print()

vehicle0 waits at s1 for 0.0, s2: for 0.0
vehicle0 charges at s1 for 2.0 using the bess and station, and for 0.0 only using station
vehicle0 charges at s2 for 4.0 using the bess and station, and for 0.0 only using station
vehicle0 reaches s1 , s2 at (3.0, 8.0)
bess energy level for vehicle0 is 50.0,50.0
vehicle0 charges s1 , s2 at (3.0, 8.0)
vehicle0 charge at s1 , s2 at (0.0, 0.0)

vehicle1 waits at s1 for 1.0, s2: for 0.0
vehicle1 charges at s1 for 3.0 using the bess and station, and for 1.0 only using station
vehicle1 charges at s2 for 2.0 using the bess and station, and for 0.0 only using station
vehicle1 reaches s1 , s2 at (4.0, 12.0)
bess energy level for vehicle1 is 40.0,30.0
vehicle1 charges s1 , s2 at (5.0, 12.0)
vehicle1 charge at s1 , s2 at (0.0, 0.0)

vehicle2 waits at s1 for 4.0, s2: for 0.0
vehicle2 charges at s1 for 2.0 using the bess and station, and for 0.0 only using station
vehicle2 charges at s2 for 4.0 using the bess and station, and for 0.0 only using station
vehi

In [235]:
print(pulp.value(cs.objective))

34.0


Code to graph outputs

In [236]:
import seaborn

In [260]:
state = {}
for vehicle in vehicles:
    state[vehicle] = []
max_len = 0
for vehicle in vehicles:
    for i in range(int(stations[0]/speed)):
        state[vehicle].append("driving")
    for j in range(int(u1[vehicle].varValue-a1[vehicle].varValue)):
        state[vehicle].append("waiting")
    for k in range(int(s1[vehicle].varValue)):
        state[vehicle].append("charging")
    for l in range(int(stations[1]/speed)):
        state[vehicle].append("driving")
    for v in range(int(u2[vehicle].varValue-a2[vehicle].varValue)):
        state[vehicle].append("waiting")
    for k in range(int(s2[vehicle].varValue)):
        state[vehicle].append("charging")
    for i in range(int(stations[2]/speed)):
        state[vehicle].append("driving")
    if len(state[vehicle])>max_len:
        max_len = len(state[vehicle])
for vehicle in vehicles:
    if len(state[vehicle]) < max_len:
        for i in range(max_len-len(state[vehicle])):
            state[vehicle].append("reached")
# state is first graph variable     max_ len is max time       
            
soc_trucks = {}
max_len_soc = 0
for vehicle in vehicles:
    soc_trucks[vehicle] = [soc_vehicle[vehicle]]
for vehicle in vehicles:
    for i in range(int(stations[0]/speed)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]-speed)
    for j in range(int(u1[vehicle].varValue-a1[vehicle].varValue)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1])
    for k in range(int(b1[vehicle].varValue)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]+ex_charging_rate_bess+charging_rate_station)
    for i in range(int(s1[vehicle].varValue-b1[vehicle].varValue)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]+charging_rate_station)
    for l in range(int(stations[1]/speed)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]-speed)
    for v in range(int(u2[vehicle].varValue-a2[vehicle].varValue)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1])
    for k in range(int(b2[vehicle].varValue)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]+ex_charging_rate_bess+charging_rate_station)
    for i in range(int(s2[vehicle].varValue-b2[vehicle].varValue)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]+charging_rate_station)
    for i in range(int(stations[2]/speed)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]-speed)
    if len(soc_trucks[vehicle])>max_len:
        max_len = len(soc_trucks[vehicle])
for vehicle in vehicles:
    if len(soc_trucks[vehicle]) < max_len:
        for i in range(max_len-len(soc_trucks[vehicle])):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1])
print(soc_trucks)

{'vehicle0': [20, 10, 0, 15, 30, 20, 10, 0, 15, 30, 45, 60, 50, 40, 30, 20, 10, 10, 10, 10, 10, 10], 'vehicle1': [20, 10, 0, 0, 15, 30, 45, 55, 45, 35, 25, 40, 55, 45, 35, 25, 15, 5, 5, 5, 5, 5], 'vehicle2': [20, 10, 0, 0, 0, 0, 0, 15, 30, 20, 10, 0, 15, 30, 45, 60, 50, 40, 30, 20, 10, 10], 'vehicle3': [20, 10, 0, 0, 0, 0, 0, 0, 15, 30, 45, 60, 50, 40, 30, 40, 50, 40, 30, 20, 10, 0]}


In [ ]:
state_bess = []
for vehicle in vehicles:
    state[vehicle] = []
max_len = 0
for vehicle in vehicles:
    for i in range(int(stations[0]/speed)):
        state[vehicle].append("driving")
    for j in range(int(u1[vehicle].varValue-a1[vehicle].varValue)):
        state[vehicle].append("waiting")
    for k in range(int(s1[vehicle].varValue)):
        state[vehicle].append("charging")
    for l in range(int(stations[1]/speed)):
        state[vehicle].append("driving")
    for v in range(int(u2[vehicle].varValue-a2[vehicle].varValue)):
        state[vehicle].append("waiting")
    for k in range(int(s2[vehicle].varValue)):
        state[vehicle].append("charging")
    for i in range(int(stations[2]/speed)):
        state[vehicle].append("driving")
    if len(state[vehicle])>max_len:
        max_len = len(state[vehicle])
for vehicle in vehicles:
    if len(state[vehicle]) < max_len:
        for i in range(max_len-len(state[vehicle])):
            state[vehicle].append("reached")
